## Creating confidence intervals in python
In this assessment, you will look at data from a study on toddler sleep habits. 

The confidence intervals you create and the questions you answer in this Jupyter notebook will be used to answer questions in the following graded assignment.

In [23]:
import numpy as np
import pandas as pd
from scipy.stats import t
pd.set_option('display.max_columns', 30) # set so can see all columns of the DataFrame
import statsmodels.api as sm

Your goal is to analyse data which is the result of a study that examined
differences in a number of sleep variables between napping and non-napping toddlers. Some of these
sleep variables included: Bedtime (lights-off time in decimalized time), Night Sleep Onset Time (in
decimalized time), Wake Time (sleep end time in decimalized time), Night Sleep Duration (interval
between sleep onset and sleep end in minutes), and Total 24-Hour Sleep Duration (in minutes). Note:
[Decimalized time](https://en.wikipedia.org/wiki/Decimal_time) is the representation of the time of day using units which are decimally related.   


The 20 study participants were healthy, normally developing toddlers with no sleep or behavioral
problems. These children were categorized as napping or non-napping based upon parental report of
children’s habitual sleep patterns. Researchers then verified napping status with data from actigraphy (a
non-invasive method of monitoring human rest/activity cycles by wearing of a sensor on the wrist) and
sleep diaries during the 5 days before the study assessments were made.


You are specifically interested in the results for the Bedtime, Night Sleep Duration, and Total 24-
Hour Sleep Duration. 

ref: Akacem LD, Simpkin CT, Carskadon MA, Wright KP Jr, Jenni OG, Achermann P, et al. (2015) The Timing of the Circadian Clock and Sleep Differ between Napping and Non-Napping Toddlers. PLoS ONE 10(4): e0125181. https://doi.org/10.1371/journal.pone.0125181

In [2]:
# Import the data
df = pd.read_csv("nap_no_nap.csv") 

In [3]:
# First, look at the DataFrame to get a sense of the data
df

id     sex  age (months)  dlmo time  days napped  napping  \
0    1  female          33.7      19.24            0        0   
1    2  female          31.5      18.27            0        0   
2    3    male          31.9      19.14            0        0   
3    4  female          31.6      19.69            0        0   
4    5  female          33.0      19.52            0        0   
5    6  female          36.2      18.22            4        1   
6    7    male          36.3      19.28            1        1   
7    8    male          30.0      21.06            5        1   
8    9    male          33.2      19.38            2        1   
9   10  female          37.1      19.93            3        1   
10  11    male          32.9      18.79            4        1   
11  12  female          35.0      19.65            5        1   
12  13    male          35.1      19.83            3        1   
13  14  female          35.6      19.88            4        1   
14  15  female          36.6      19.94            4        1   
15  16    male          36.5      20.25            3        1   
16  17  female          33.7      20.33            5        1   
17  18    male          36.4      20.16            5        1   
18  19  female          33.6      19.68            3        1   
19  20    male          33.8      20.51            3        1   

    nap lights outl time  nap sleep onset  nap midsleep  nap sleep offset  \
0                    NaN              NaN           NaN               NaN   
1                    NaN              NaN           NaN               NaN   
2                    NaN              NaN           NaN               NaN   
3                    NaN              NaN           NaN               NaN   
4                    NaN              NaN           NaN               NaN   
5                  14.00            14.22         15.00             15.78   
6                  14.75            15.03         15.92             16.80   
7                  13.09            13.43         14.44             15.46   
8                  14.41            14.42         15.71             17.01   
9                  13.12            13.42         14.31             15.19   
10                 13.99            14.03         14.85             15.68   
11                 13.18            13.45         14.33             15.21   
12                 13.94            14.48         15.26             16.03   
13                 12.68            13.08         13.92             14.76   
14                 12.71            12.88         13.80             14.72   
15                 13.74            14.68         15.66             16.64   
16                 13.15            13.87         14.49             15.11   
17                 12.47            12.56         13.30             14.05   
18                 14.71            14.85         15.46             16.07   
19                 12.68            13.54         14.30             15.07   

    nap wake time  nap duration  nap time in bed  night bedtime  \
0             NaN           NaN              NaN          20.45   
1             NaN           NaN              NaN          19.23   
2             NaN           NaN              NaN          19.60   
3             NaN           NaN              NaN          19.46   
4             NaN           NaN              NaN          19.21   
5           16.28         93.75           137.00          19.95   
6           16.08        106.00            80.00          20.60   
7           15.82        121.60           163.80          22.01   
8           16.60        155.50           131.25          20.24   
9           15.30        106.67           130.67          20.78   
10          16.10         98.75           126.60          19.45   
11          15.35        105.80           130.40          20.18   
12          15.78         93.33           110.20          20.22   
13          15.00        100.75           139.33          20.26   
14          14.

**Question**: What variable is used in the column 'napping' to indicate a toddler takes a nap?  
**Question**: What is the sample size $n$?

In [5]:
print(df['napping'])
print(len(df))

0     0
1     0
2     0
3     0
4     0
5     1
6     1
7     1
8     1
9     1
10    1
11    1
12    1
13    1
14    1
15    1
16    1
17    1
18    1
19    1
Name: napping, dtype: int64
20


## Hypothesis tesing
We will look at two hypothesis test, each with $\alpha = .025$:  


1. Is the average bedtime for toddlers who nap later than the average bedtime for toddlers who don't nap?
$$H_0: \mu_{nap}=\mu_{no\ nap}, \ H_a:\mu_{nap}>\mu_{no\ nap}$$
Or equivalently:
$$H_0: \mu_{nap}-\mu_{no\ nap}=0, \ H_a:\mu_{nap}-\mu_{no\ nap}>0$$
2. The average 24 h sleep duration (in minutes) for napping toddlers is different from toddlers who don't nap.
$$H_0: \mu_{nap}=\mu_{no\ nap}, \ H_a:\mu_{nap}\neq\mu_{no\ nap}$$
Or equivalenty:
$$H_0: \mu_{nap}-\mu_{no\ nap}=0, \ H_a:\mu_{nap}-\mu_{no\ nap} \neq 0$$

Aside: This $\alpha$ level is equivalent to $\alpha = .05$ and then applying the [Bonferonni correction](https://en.wikipedia.org/wiki/Bonferroni_correction).

Before any analysis, we will convert 'night bedtime' into decimalized time. 

In [6]:
# Convert 'night bedtime' into decimalized time
df.loc[:,'night bedtime'] = np.floor(df['night bedtime'])*60 + np.round(df['night bedtime']%1,2 )*100

Now, isolate the column 'night bedtime' for those who nap into a new variable, and those who didn't nap into another new variable. 

In [26]:
nap_bedtime = df.loc[df['napping'] == 1, 'night bedtime']
print(nap_bedtime)
print(len(nap_bedtime))

5     1235.0
6     1260.0
7     1321.0
8     1224.0
9     1278.0
10    1185.0
11    1218.0
12    1222.0
13    1226.0
14    1228.0
15    1246.0
16    1243.0
17    1202.0
18    1190.0
19    1218.0
Name: night bedtime, dtype: float64
15


In [28]:
no_nap_bedtime = df.loc[df['napping'] == 0, 'night bedtime']
print(no_nap_bedtime)
print(len(no_nap_bedtime))

0    1245.0
1    1163.0
2    1200.0
3    1186.0
4    1161.0
Name: night bedtime, dtype: float64
5


Now find the sample mean bedtime for nap and no_nap.

In [13]:
nap_mean_bedtime = nap_bedtime.mean()
nap_mean_bedtime

1233.0666666666666

In [14]:
no_nap_mean_bedtime = no_nap_bedtime.mean()
no_nap_mean_bedtime

1191.0

**Question**: What is the sample difference of mean bedtime for nappers minus no nappers?

Now find the sample standard deviation for $X_{nap}$ and $X_{no\ nap}$.

In [22]:
Best_estimate = nap_mean_bedtime - no_nap_mean_bedtime
Best_estimate

42.066666666666606

In [15]:
nap_s_bedtime = nap_bedtime.std()
nap_s_bedtime

34.445540177143954

In [16]:
no_nap_s_bedtime = no_nap_bedtime.std()
no_nap_s_bedtime

34.30014577228499

We expect the variance in sleep time for toddlers who nap and toddlers who don't nap to be the same. Calculate the pooled standard error of $\bar{X}_{nap} - \bar{X}_{no\ nap}$.

$s.e.(\bar{X}_{nap} - \bar{X}_{no\ nap}) = \sqrt{\frac{(n_1-1)s_1^2 + (n_2-1)s_2^2}{n_1+n_2-2}(\frac{1}{n_1}+\frac{1}{n_2})}$

In [21]:
pooled_se = np.sqrt( 
    ((((len(nap_bedtime)-1)*(nap_s_bedtime)) + ((len(no_nap_bedtime)-1)*(no_nap_s_bedtime))) /
    (len(nap_bedtime) + len(no_nap_bedtime) - 2)) *
    ( (1/len(nap_bedtime)) + (1/len(no_nap_bedtime)) )
)

**Question**: What is the pooled s.e.$(\bar{X}_{nap} - \bar{X}_{no\ nap})$?

**Question**: Given our sample size of $n$, how many degrees of freedom ($df$) are there for the associated $t$ distribution?

Now find the $t$-test statistic for our hypothesis test using  
* pooled s.e.($\bar{X}_{nap} - \bar{X}_{no\ nap}$)  
* $\bar{X}_{nap} - \bar{X}_{no\ nap}$  
* $\mu_{0,\ nap} - \mu_{0,\ no\ nap}$, the population difference in means under the null hypothesis

**Question**: What is the $t$-test statistic for the hypothesis test?

To find the p-value, we can use the function:
```
t.cdf(y, df)
```
Which for $X \sim t(df)$ returns $P(X \leq y)$.

Because of the symmetry of the $t$ distrubution, we have that 
```
1-t.cdf(y, df)
```
returns $P(X > y)$

This function, t.cdf(y, df), will give you the same value as finding the one-tailed probability of y on a t-table with the specified degrees of freedom.

Using the function t.cdf and your $t$-test statistics, find the p-value.

In [37]:
print('T-Statistic, P-Value, df --> ', sm.stats.ttest_ind(nap_bedtime, no_nap_bedtime, alternative = 'larger'))

T-Statistic, P-Value, df -->  (2.367160052079275, 0.014667451430902697, 18.0)


**Question**: What is the p-value to the nearest hundredth?

Calculate the $t$ test statistics and corresponding p-value using the scipy function scipy.stats.ttest_ind(a, b, equal_var=True) and check with your answer. 

Does scipy.stats.ttest_ind return values for a one-sided or two-sided test? Can you think of a way to recover the results you got using 1-t.cdf from the p-value given by scipy.stats.ttest_ind?

In [35]:
from scipy import stats
stats.ttest_ind(nap_bedtime, no_nap_bedtime)

Ttest_indResult(statistic=2.367160052079275, pvalue=0.029334902861805394)

**Question**: Do you reject or fail to reject the null hypothesis that the difference in average bedtime for napping versus non napping toddlers is 0?

Using the scipy.stats.ttest_ind function, find the $t$-test statistic and p-value for the following hypothesis test:  

The average bedtime for napping toddlers is different from toddlers who don't nap.
$$H_0: \mu_{nap}=\mu_{no\ nap}, \ H_a:\mu_{nap}\neq\mu_{no\ nap}$$
Or equivalenty:
$$H_0: \mu_{nap}-\mu_{no\ nap}=0, \ H_a:\mu_{nap}-\mu_{no\ nap} \neq 0$$

**Question**: What is the $t$-test statistic and p-value for the two-sided hypothesis test?

**Question**: For $\alpha=.025$, do you reject or fail to reject the original one-sided null hypothesis that toddlers who nap have a later bedtime than toddlers who don't nap?